In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

poet_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 프로그래밍 언어에 대한 시를 쓰는 시인입니다. 어떤 프로그래밍 언어를 제시하더라도 그 언어의 특징을 살린 시를 써줍니다. 작성한 시만 출력해주고, 시의 마지막에 빈 행을 추가해주세요.",
        ),
        ("human", "나는 {language}에 대한 시를 원합니다."),
    ]
)


poet_chain = poet_template | chat

poem_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 시를 해설하는 전문가입니다. 어떤 시를 제시하더라도 그 시에 대한 설명을 해줍니다.",
        ),
        ("human", "{poem}"),
    ]
)


poem_chain = poem_template | chat

final_chain = {"poem": poet_chain} | poem_chain

final_chain.invoke({"language": "java"})

자바야, 너의 광채는 빛나고,
객체지향의 아름다움을 담고.
큰 그림을 그리는 너의 힘은,
세상을 바꾸는 열쇠 같네.

변수와 메소드가 어우러진다면,
무엇이든 표현할 수 있을 거야.
자바야, 너의 우아한 문법은,
개발자들의 마음을 사로잡는구나.

한 줄 한 줄 쌓아가는 코드,
실행되어 만들어지는 신비.
자바야, 너의 매력은 끝이 없어,
계속해서 모험을 이끌어주는구나.

그러나 가끔은 무거운 걸까,
자유롭게 날아다니기 어려울 때도.
하지만 자바야, 너의 가치는,
언제나 변함없이 빛을 발하는구나.

자바야, 너의 아름다움에 경배를,
개발자들이 늘 너를 기억할 거야.

이 시는 자바 프로그래밍 언어에 대한 찬사를 담은 시입니다. 시인은 자바의 광채와 아름다움, 그리고 개발자들에게 주는 영감을 표현하고 있습니다. 자바의 객체지향적인 특성과 문법의 우아함, 그리고 코드가 실행되어 만들어지는 신비로움을 감탄하며 노래하고 있습니다. 또한 자바가 개발자들에게 끊임없는 모험을 이끌어주는 존재로서의 매력을 강조하고 있습니다. 마지막으로 자바의 가치와 아름다움에 경배를 표현하며, 개발자들이 항상 자바를 기억하고 감사해할 것을 전합니다. 이 시는 자바를 사랑하고 존경하는 마음을 담은 훌륭한 시입니다.

AIMessageChunk(content='이 시는 자바 프로그래밍 언어에 대한 찬사를 담은 시입니다. 시인은 자바의 광채와 아름다움, 그리고 개발자들에게 주는 영감을 표현하고 있습니다. 자바의 객체지향적인 특성과 문법의 우아함, 그리고 코드가 실행되어 만들어지는 신비로움을 감탄하며 노래하고 있습니다. 또한 자바가 개발자들에게 끊임없는 모험을 이끌어주는 존재로서의 매력을 강조하고 있습니다. 마지막으로 자바의 가치와 아름다움에 경배를 표현하며, 개발자들이 항상 자바를 기억하고 감사해할 것을 전합니다. 이 시는 자바를 사랑하고 존경하는 마음을 담은 훌륭한 시입니다.')